In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline


In [12]:
import numpy
import scipy
import math
import matplotlib
import matplotlib.dates as mpd
import pylab as plt
import datetime as dtm
import pytz
import multiprocessing as mpp
import pickle
import sys
import os
import subprocess
#import lmod
# lmod.load('system')
# lmod.load('texlive')
# lmod.
#
# TODO: phase out unreferenced hpc_lib calls...
#import hpc_lib
#import hpc_reports
#
def running_mean(X,n=10):
    return (numpy.cumsum(numpy.insert(X,0,0))[n:] - numpy.cumsum(numpy.insert(X,0,0))[:-n])/n
#

## Realtime Monitoring tools

### General objective:
- Some tools to do realtime monitoring of SERC usage
- Nominally more versatile than that; could be SERC, users, groups, etc. 
- Design some metrics to indicate excessive usage; send alerts; make pretty graphs.

Possible starting query:
        squeue -p serc --Format=jobid,jobarrayid,partition,name,userid,timeused,timeleft,numnodes,nodelist,numcpus,state

Use this to identify:
- An individual user or group using >1/3 of the cluster for >1 hour (or so)
- Generally, times of high impact (lots of running and pending jobs)

Note that it is not trivial to estimate this. Loads of jobs pending is not significant, for example, if they have short runtimes. What we really want to catch is a scenario where a job array (or similar) picks up a bunch of idle resources (eg, overnight when usage is lower) for >24 hour runtime jobs.


In [13]:
# srun_fields = ['jobid', 'jobarrayid', 'partition', 'name', 'userid', 'timeused', 'timeleft', 'numnodes',
#              'nodelist', 'numcpus', 'state']
# srun_partitions = ['serc']
# srun_users  = []
# srun_groups = []
srun_fields = {'--Format': ['jobid', 'jobarrayid', 'partition', 'name', 'userid', 'timeused', 
                              'timeleft', 'numnodes',              'nodelist', 'numcpus', 'state'],
              '--partition': ['serc']
              }
#
sinfo_str = 'squeue '
for ky,vl in srun_fields.items():
    delim=' '
    if ky.startswith('--'):
        # long format
        delim='='
    #
    sinfo_str = '{} {}{}{}'.format(sinfo_str, ky, delim, ','.join(vl))
    
#
print('*** sinfo_str: {}'.format(sinfo_str))
print('*** sinfo_ary: {}'.format(sinfo_str.split()))
#

*** sinfo_str: squeue  --Format=jobid,jobarrayid,partition,name,userid,timeused,timeleft,numnodes,nodelist,numcpus,state --partition=serc
*** sinfo_ary: ['squeue', '--Format=jobid,jobarrayid,partition,name,userid,timeused,timeleft,numnodes,nodelist,numcpus,state', '--partition=serc']


In [18]:
squeue_output = subprocess.run(srun_str.split(), stdout=subprocess.PIPE).stdout.decode().split('\n')
cols = squeue_output.readline()
data_dct = {cl:[] for cl in cols}
#
# for now, let's just make a dict with the columns... or use a f***ing PANDAs DF? most of the columns
#. are text-like values, so a recarray or structured numpy array is probably not a good idea... though
#  it's not that difficult to format columns...
data = []
for rw in squeue_output:
    data += [rw.split()]
#
data_df = 

**  JOBID               JOBID               PARTITION           NAME                UID                 TIME                TIME_LEFT           NODES               NODELIST            CPUS                STATE               
**  46735452            46735452            owners,serc,gpu     train               313386              0:00                1-00:00:00          1                                       24                  PENDING             
**  46683021            46683021            serc                check_scratch_cron  337769              0:00                5:00                1                                       1                   PENDING             
**  46660783            46660783            serc                check_scratch_cron  351294              0:00                5:00                1                                       1                   PENDING             
**  46723185            46723185_[1-20]     serc                mima_run            402579          